In [1]:
# import random as rnd
# import secrets as scr

from datetime import datetime
from random import SystemRandom

rnd = SystemRandom()


class Lotteria:
    def __init__(self,
                 max_numbers=90,
                 max_extra=90,
                 numbers=6,
                 extra=1):
        self.max_numbers = max_numbers
        self.max_extra = max_extra
        self.numbers = numbers
        self.extra = extra

    @staticmethod
    def pallottoliere(max_nums):
        while True:
            yield rnd.randint(1, max_nums)

    @staticmethod
    def choice(nums, max_nums):
        whole_numbers = list(range(1, max_nums+1))
        
        def extraction():
            rnd.shuffle(whole_numbers)
            single = rnd.choice(whole_numbers)
            whole_numbers.remove(single)

            return single

        return [extraction() for _ in range(nums)]

    @staticmethod
    def sample(nums, max_nums):
        whole_numbers = list(range(1, max_nums+1))

        return rnd.sample(whole_numbers, k=nums)

    def int(self, nums, max_nums):
        combo = set()
        while len(combo) < nums:
            combo.add(next(self.pallottoliere(max_nums)))

        return combo

    @staticmethod
    def many_(backend, nums, max_nums):
        samples = [backend(nums, max_nums) for _ in range(max_nums)]
        rnd.shuffle(samples)

        return rnd.choice(samples)

    @property
    def backend(self):
        return self._backend

    @property
    def many(self):
        return self._many

    def __call__(self, backend='sample', many=True):
        self._backend = backend
        self._many = many

        backend = eval(backend,
                       {'__builtins__': {}},
                       {'choice': self.choice, 'int': self.int, 'sample': self.sample})

        if self._many:
            combo = self.many_(backend,
                               self.numbers,
                               self.max_numbers)

            if self.extra:
                extra = self.many_(backend,
                                   self.extra,
                                   self.max_extra)

        else:
            combo = backend(self.numbers, self.max_numbers)

            if self.extra:
                extra = backend(self.extra, self.max_extra)

        now = datetime.now().strftime("%d/%m/%Y %H:%M")

        print(f'Estrazione del: {now}', '\nNumeri Estratti:', *sorted(combo))
        if self.extra:
            print('Superstar:', *sorted(extra))  # type: ignore

In [2]:
eurojack = Lotteria(max_numbers=50, max_extra=12, numbers=5, extra=2)

In [3]:
eurojack()

Estrazione del: 04/06/2023 22:37 
Numeri Estratti: 4 9 21 31 42
Superstar: 4 10


In [4]:
superenalotto = Lotteria(max_numbers=90, max_extra=90, numbers=6, extra=0)

In [5]:
superenalotto(backend='choice')

Estrazione del: 04/06/2023 22:37 
Numeri Estratti: 18 50 52 65 68 72


In [6]:
for b in ['choice', 'int', 'sample']:
    print('######')
    eurojack(backend=b, many=False)
    print('backend:', eurojack.backend)

print('\n################# many ####################\n')

for b in ['choice', 'int', 'sample']:
    print('######')
    eurojack(backend=b, many=True)
    print('backend:', eurojack.backend)

######
Estrazione del: 04/06/2023 22:37 
Numeri Estratti: 12 30 36 37 48
Superstar: 10 12
backend: choice
######
Estrazione del: 04/06/2023 22:37 
Numeri Estratti: 1 9 24 31 40
Superstar: 8 10
backend: int
######
Estrazione del: 04/06/2023 22:37 
Numeri Estratti: 15 20 41 44 50
Superstar: 7 12
backend: sample

################# many ####################

######
Estrazione del: 04/06/2023 22:37 
Numeri Estratti: 14 16 26 43 46
Superstar: 2 3
backend: choice
######
Estrazione del: 04/06/2023 22:37 
Numeri Estratti: 17 33 36 40 41
Superstar: 1 2
backend: int
######
Estrazione del: 04/06/2023 22:37 
Numeri Estratti: 3 7 8 24 46
Superstar: 3 12
backend: sample


In [7]:
superenalotto.extra=1
superenalotto(backend='choice', many=False)

Estrazione del: 04/06/2023 22:37 
Numeri Estratti: 25 29 41 81 86 88
Superstar: 19


In [8]:
superenalotto(backend='int')

Estrazione del: 04/06/2023 22:37 
Numeri Estratti: 15 56 62 73 75 82
Superstar: 54


In [9]:
superenalotto.extra

1

In [10]:
superenalotto.many

True

In [11]:
winForLife = Lotteria(20, 20, 10, 1)
winForLife(backend='int', many=False)

Estrazione del: 04/06/2023 22:37 
Numeri Estratti: 1 3 5 6 7 8 12 15 16 19
Superstar: 9


In [12]:
# trying to set backend via attribute: fails as expected
winForLife.backend = 'choice'

AttributeError: property 'backend' of 'Lotteria' object has no setter